In [68]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests 
from bs4 import BeautifulSoup
import time
import math
import pandas as pd
from google.colab import drive
drive.mount('drive')

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (151 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
while True:
  driver.get('https://www.google.com/search?q=us+election+results&oq=us+election+results&aqs=chrome.0.69i59j0i131i433l6j69i60.8640j0j9&sourceid=chrome&ie=UTF-8')
  timeout = 3
  try:
      element_present = EC.presence_of_element_located((By.ID, '_83GlX73aK5_G4-EPqtC9uAQ95'))
      WebDriverWait(driver, timeout).until(element_present)
  except TimeoutException:
      print("Timed out waiting for page to load")
  finally:
      print("Page loaded")
      userid_element = driver.find_elements_by_xpath('//div[@Style="width:172px;text-align:center"]')[0]
      userid_element.click()
      time.sleep(1)
      userid_element = driver.find_elements_by_xpath('//span[@class="PLaf5"]')
      userid_element1 = driver.find_elements_by_xpath('//span[@class="E2WDKf"]')


      data = []
      for i in userid_element:
        if i.text != "":
          data.append(i.text)
      for i in userid_element1:
        if i.text != "":
          data.append(int(i.text.replace('% reporting','')))
      for i in range(int(len(data)/2)):
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[1].text.replace(',','')))
        data.append(int(driver.find_elements_by_xpath('//div[@class="oLhH0e"]')[i].find_elements_by_tag_name('span')[3].text.replace(',','')))



      print(data)
      rawData = []
      for i in range(int(len(data)/4)):
        d = []
        for j in range(2):
          d.append(data[i+j*6])
        d.append(data[12+i*2])
        d.append(data[13+i*2])
        rawData.append(d)
      print(rawData)

      
      info =[]
      data = rawData
      for i in range(len(data)):
        raw = data[i]
        d =[]
        d.append(raw[0])
        diff = raw[2] - raw[3]
        percent = raw[1]/100.0
        uncountedVotes = math.floor((raw[2]+raw[3])*(1-percent)/percent)
        if diff < 0:
          voteNeeded = math.ceil(((uncountedVotes + diff)/2) -diff)
        else:
          voteNeeded = math.ceil(((uncountedVotes - diff)/2))

        percentNeeded = voteNeeded/(uncountedVotes*1.0)
        d.append(str(raw[1])+"%")
        d.append(raw[2])
        d.append(raw[3])
        if percentNeeded < 1 and voteNeeded > 0:
          d.append(uncountedVotes)
          d.append(voteNeeded)
          d.append(str(math.ceil(percentNeeded*1000)/10.0)+"%")
          d.append(uncountedVotes+1-voteNeeded)
          d.append(str(math.ceil((((uncountedVotes+1-voteNeeded)/uncountedVotes)*1000))/10.0)+"%")
          info.append(d)
          print(d)
        elif voteNeeded <= 0:
          d.append(uncountedVotes)
          d.append('Already Won')
          d.append('Already Won')
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          info.append(d)
          print(d)
        elif percentNeeded >= 1:
          d.append(uncountedVotes)
          d.append('Impossible to Win in This World')
          d.append('Impossible to Win in This World')
          d.append('Already Won')
          d.append('Already Won')
          info.append(d)
          print(d)

      print(info)
      df = pd.DataFrame (info,columns=['State Name','Percentage of Counted Votes','Counted Votes for Biden','Counted Votes for Trump','Uncounted Votes','Vote Biden Needed to Win','Percentage of Uncounted Vote Biden Needed to Win','Vote Trump Needed to Win','Percentage of Uncounted Vote Biden Needed to Win'])
      mainPath =r'drive/My Drive/New Folder'
      csvPath =mainPath+"/"+"USElectionSwingState.csv"
      df.to_csv(csvPath)
      time.sleep(120)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


Timed out waiting for page to load
Page loaded
['Georgia', 'Nevada', 'North Carolina', 'Pennsylvania', 'Arizona', 'Florida', 99, 87, 99, 98, 90, 99, 2455426, 2451191, 627104, 606967, 2655629, 2732302, 3315745, 3301204, 1565200, 1525800, 5284453, 5658847]
[['Georgia', 99, 2455426, 2451191], ['Nevada', 87, 627104, 606967], ['North Carolina', 99, 2655629, 2732302], ['Pennsylvania', 98, 3315745, 3301204], ['Arizona', 90, 1565200, 1525800], ['Florida', 99, 5284453, 5658847]]
['Georgia', '99%', 2455426, 2451191, 49561, 22663, '45.8%', 26899, '54.3%']
['Nevada', '87%', 627104, 606967, 184401, 82132, '44.6%', 102270, '55.5%']
['North Carolina', '99%', 2655629, 2732302, 54423, 'Impossible to Win in This World', 'Impossible to Win in This World', 'Already Won', 'Already Won']
['Pennsylvania', '98%', 3315745, 3301204, 135039, 60249, '44.7%', 74791, '55.4%']
['Arizona', '90%', 1565200, 1525800, 343444, 152022, '44.3%', 191423, '55.8%']
['Florida', '99%', 5284453, 5658847, 110538, 'Impossible to Wi